# Cluster Artifacts based on Dynamic Centroid

In [21]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.insert(0, "..")
import os
import dbutils
import math
import numpy as np
from cgmcore import modelutils, utils
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import progressbar
import psycopg2
import glob
import pickle
import cv2
import posenet
import tensorflow as tf
import time
from tqdm import tqdm



# for this experiment we choose an arbitrary model for the clustering
model = "20191015-1725_464-116weight_v5"




In [57]:
db_connector = dbutils.connect_to_main_database()


# get a list of all unique qr_codes
sql_statement   = "SELECT DISTINCT artifact.qr_code FROM artifact  ORDER BY qr_code ASC;"
unique_qr_codes = db_connector.execute(sql_statement, fetch_all=True)
unique_qr_codes = [x[0] for x in unique_qr_codes]

# remove first two elements due to crap content
unique_qr_codes.pop(0)
unique_qr_codes.pop(0)

print(unique_qr_codes)


Loading /data/home/cpfitzner/notebooks/cgm-ml/cgm_database/dbconnection.json...
Host:    cgm-dev.postgres.database.azure.com
DB-name: cgm_ml_db
['MH_WHH_0001', 'MH_WHH_0002', 'MH_WHH_0003', 'MH_WHH_0004', 'MH_WHH_0006', 'MH_WHH_0007', 'MH_WHH_0008', 'MH_WHH_0009', 'MH_WHH_0010', 'MH_WHH_0011', 'MH_WHH_0012', 'MH_WHH_0013', 'MH_WHH_0014', 'MH_WHH_0015', 'MH_WHH_0016', 'MH_WHH_0017', 'MH_WHH_0018', 'MH_WHH_0019', 'MH_WHH_0022', 'MH_WHH_0026', 'MH_WHH_0027', 'MH_WHH_0028', 'MH_WHH_0029', 'MH_WHH_0030', 'MH_WHH_0031', 'MH_WHH_0032', 'MH_WHH_0033', 'MH_WHH_0034', 'MH_WHH_0035', 'MH_WHH_0036', 'MH_WHH_0037', 'MH_WHH_0039', 'MH_WHH_0041', 'MH_WHH_0042', 'MH_WHH_0043', 'MH_WHH_0044', 'MH_WHH_0045', 'MH_WHH_0046', 'MH_WHH_0047', 'MH_WHH_0048', 'MH_WHH_0049', 'MH_WHH_0052', 'MH_WHH_0053', 'MH_WHH_0054', 'MH_WHH_0056', 'MH_WHH_0063', 'MH_WHH_0075', 'MH_WHH_0076', 'MH_WHH_0077', 'MH_WHH_0081', 'MH_WHH_0082', 'MH_WHH_0083', 'MH_WHH_0095', 'MH_WHH_0096', 'MH_WHH_0097', 'MH_WHH_0102', 'MH_WHH_0104', 

In [89]:
for qr_code in unique_qr_codes: 
    print(qr_code)
   
    select_sql_statement = ""
    select_sql_statement += "SELECT  height FROM measure m"
    select_sql_statement += " INNER JOIN artifact a ON m.id = a.measure_id"
   # select_sql_statement += " INNER JOIN measure m ON m.id = a.id"
    select_sql_statement += " WHERE a.qr_code = '{}'".format(qr_code)
   # select_sql_statement += " AND a.type = 'pcd'" 
    results = db_connector.execute(select_sql_statement, fetch_all=True)
    print (results)
    
    
    
    select_sql_statement = ""
    select_sql_statement += "SELECT  value FROM artifact_result ar"
    select_sql_statement += " INNER JOIN artifact a ON a.id = ar.artifact_id"
   # select_sql_statement += " INNER JOIN measure m ON m.id = a.id"
    select_sql_statement += " WHERE qr_code = '{}'".format(qr_code)
   # select_sql_statement += " AND a.type = 'pcd'" 
    results = db_connector.execute(select_sql_statement, fetch_all=True)
    print (results)
    
    
    
    

MH_WHH_0001
[(99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,), (99.0,

[(69.9985,), (70.1059,), (70.0298,), (70.005,), (70.0955,), (70.2672,), (70.7419,), (69.0547,), (68.8958,), (69.4731,), (72.5238,), (77.9081,), (76.8167,), (76.6519,), (80.9992,), (71.5235,), (71.7102,), (65.2532,), (65.5084,), (64.9458,), (68.0287,), (70.752,), (71.9515,), (75.6809,), (77.4826,), (73.0157,), (66.238,), (65.626,), (69.4852,), (70.4167,), (66.9954,), (70.2236,)]
MH_WHH_0006
[]
[]
MH_WHH_0007


KeyboardInterrupt: 

In [6]:


select_sql_statement = ""
select_sql_statement += "SELECT COUNT(*) FROM artifact_quality"
# select_sql_statement += " WHERE artifact_id='{}'".format(artifact_id)
# select_sql_statement += " AND type='{}'".format(db_type)
# select_sql_statement += " AND key='{}'".format(db_key)
results = db_connector.execute(select_sql_statement, fetch_one=True)[0]

print(results)
    

Loading /data/home/cpfitzner/notebooks/cgm-ml/cgm_database/dbconnection.json...
Host:    cgm-dev.postgres.database.azure.com
DB-name: cgm_ml_db
3107140


In [49]:
level = "scan"
level = "artifact"

results = db_connector.execute(select_sql_statement, fetch_all=True)



#target_key = model_details["dataset_parameters"]["output_targets"][0]

select_sql_statement += "SELECT COUNT(*) FROM artifact_result"
select_sql_statement += " WHERE artifact_id='{}'".format(artifact_id)
select_sql_statement += " AND model_name='{}'".format(model)
select_sql_statement += " AND target_key='weight'"

print (results)


# SQL statement for scan level.
if level == "scan":
    select_sql_statement = ""
    select_sql_statement += "SELECT AVG(aq.value) FROM artifact_quality aq"
    select_sql_statement += " INNER JOIN artifact a ON a.id = aq.artifact_id"
    select_sql_statement += " WHERE aq.key='mae'"
    select_sql_statement += " AND aq.type='{}'".format(model)
    select_sql_statement += " GROUP BY a.qr_code, a.create_date"
    select_sql_statement += ";"

# SQL statement for artifact level.
elif level == "artifact":
    select_sql_statement = ""
    select_sql_statement += "SELECT value FROM artifact_quality" 
    select_sql_statement += " WHERE type='{}' AND key='mae'".format(model)
    select_sql_statement += ";"

# Retrieve results from DB.
#results = db_connector.execute(select_sql_statement, fetch_all=True)
results = [x[0] for x in results]

#print(results)

NameError: name 'artifact_id' is not defined